In [1]:
# import libraries
import numpy as np
import pandas as pd



### Text handling module

In [2]:
from process import texthandling as th
a = pd.DataFrame({'id':   [1,2,3],
                        'name': ['Mr.Tanaka', 'Ms.Suzuki', 'Mr.Black']})
b = ['Mr', 'Ms', 'Tanaka']

th.FlagWords(a['name'], b)